In [6]:
import json
import pandas as pd
import dill as pickle

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')

        return

In [7]:
#######################
# LigthGBM Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm
import random
import json

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id']

cate_cols = ['store_id_encoded', 'area_name', 'city', 'genre_name']
cate_feats = ['dow', 'hol_days', 'day', 'pom', 'prev_is_holiday', 'next_is_holiday', 
              'wom', 'woy', 'is_weekends', 'holiday_flg', 'month', 'is_up_corner']
for mod in ['air', 'hpg']:
    cate_feats.extend(['%s_%s' % (mod, c) for c in cate_cols])
    
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir
kfold = 5
strategy = 'lgb_l2'
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []

for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    holdout = pd.read_csv('%s/holdout.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    test = pd.read_csv('%s/test.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)

    #
    valid['fold'] = fold
    valid_dfs.append(valid)
    holdout_dfs.append(holdout)
    test_dfs.append(test)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
##### model selection with CV
# score
holdout_score = .0
# parameters
params = {
    "boosting": "gbdt",
    "objective": "regression_l2",
    "lambda_l2": [200, 80, 20],
    "learning_rate": [0.4, 0.2, 0.1],
    "max_depth": [12, 16, 20],
    
#     "lambda_l2": [20],
#     "learning_rate": [0.2],
#     "max_depth": [16],
#     "device": "gpu",
    
    "num_iterations": 150,
    "min_data_in_leaf": 20,
    'num_leaves': 255,

    "feature_fraction": 0.90,
    "bagging_fraction": 0.85,
    "bagging_freq": 20,
    "min_hessian": 0.001,

    "max_bin": 63,
}
total_cols = [c for c in TrainData.columns if c not in drop_cols]
K = int(0.6 * len(total_cols))
N = 100


start = time.time()
for idx in range(N): 
    selected_cols = [total_cols[i] for i in sorted(random.sample(range(len(total_cols)), K))]
    OutputDir = '%s/MM2/l1/%s' % (DataBaseDir, idx)
    if(os.path.exists(OutputDir) == False):
        os.makedirs(OutputDir)
    # save feature space
    with open('%s/sub_feats.txt' % OutputDir, 'w') as o_file:
        for feat in selected_cols:
            o_file.write('%s\n' % feat)
    o_file.close()
    ##
    BestParmas = {}
    BestScore = 1.0
    cv_score = .0
    for l2 in params['lambda_l2']:
        for lr in params['learning_rate']:
            for depth in params['max_depth']:
                cv_rmsle = .0
                for fold in range(kfold):
                    FoldData = {
                        'train': TrainData[TrainData['fold'] != fold],
                        'valid': TrainData[TrainData['fold'] == fold]
                    }
                    d_cv = lightgbm.Dataset(FoldData['train'][selected_cols], 
                                        label= FoldData['train']['visitors'], 
                                        #max_bin= params['max_bin'], 
                                        silent= True, 
                                        free_raw_data= True)
                    param = {
                        'boosting': 'gbdt',
                        'objective': 'regression_l2',

                        'lambda_l2': l2,
                        'learning_rate': lr,
                        'max_depth': depth,

                        'num_iterations': params['num_iterations'],
                        'feature_fraction': params['feature_fraction'],
                        'bagging_fraction': params['bagging_fraction'],
                        'bagging_freq': params['bagging_freq'],
                        'min_hessian': params['min_hessian'],
                        'max_bin': params['max_bin'],
                    }
                    model = lightgbm.train(param, d_cv)
                    FoldData['valid'][strategy] = model.predict(FoldData['valid'][selected_cols])
                    rmsle_valid = RMSLE(FoldData['valid']['visitors'], FoldData['valid'][strategy])
                    cv_rmsle += rmsle_valid
                cv_rmsle /= kfold
                if(cv_rmsle < BestScore):
                    BestScore = cv_rmsle
                    BestParmas['lambda_l2'] = l2
                    BestParmas['learning_rate'] = lr
                    BestParmas['max_depth'] = depth
                end = time.time()
                print('running for params, l2 %s, lr %s, depth %s done, cv score %.5f, time elapsed %.2fs' % (l2, lr, depth, cv_rmsle, (end - start)))
    end = time.time()
    print('grid search done, time elapsed %.2fs' % (end - start))
    print('best params, %s' % BestParmas)
    ## retrain and store
    cv_score = .0
    holdout_score = .0
    for fold in range(kfold):
        FoldData = {
            'train': TrainData[TrainData['fold'] != fold],
            'valid': TrainData[TrainData['fold'] == fold],
            'holdout': holdout_dfs[fold],
            'test': test_dfs[fold]
        }
        # train
        d_cv = lightgbm.Dataset(FoldData['train'][selected_cols], 
                            label= FoldData['train']['visitors'].values, 
                            #max_bin= params['max_bin'], 
                            silent= True, 
                            free_raw_data= True)
        param = {
            'boosting': 'gbdt',
            'objective': 'regression_l2',
                
            'lambda_l2': BestParmas['lambda_l2'],
            'learning_rate': BestParmas['learning_rate'],
            'max_depth': BestParmas['max_depth'],
                        
            'num_iterations': params['num_iterations'],
            'feature_fraction': params['feature_fraction'],
            'bagging_fraction': params['bagging_fraction'],
            'bagging_freq': params['bagging_freq'],
            'min_hessian': params['min_hessian'],
            'max_bin': params['max_bin'],
        }
        model = lightgbm.train(param, d_cv)
        # for valid
        FoldData['valid'][strategy] = model.predict(FoldData['valid'][selected_cols])
        rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
        cv_score += rmsle_valid
        # for holdout
        FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][selected_cols])
        rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
        holdout_score += rmsle_holdout
        # for test
        FoldData['test'][strategy] = model.predict(FoldData['test'][selected_cols])

        print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
        #### output
        FoldOutputDir = '%s/kfold/%s' % (OutputDir, fold)
        if(os.path.exists(FoldOutputDir) == False):
            os.makedirs(FoldOutputDir)
        for mod in FoldData.keys():
            if(mod == 'train'):
                continue
            OutCols = []
            if(mod == 'test'):
                OutCols.append('id')
            OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
            OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
            OutFoldData = FoldData[mod][OutCols]
            OutFoldData.to_csv(OutputFile, index= False)
    
    cv_score /= kfold # Average valid set predictions
    holdout_score /= kfold # Average holdout set predictions

    end = time.time()
    print('\n======================')
    print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, (end - start)))
    print('======================\n')
    
    with open('%s/result.txt' % OutputDir, 'w') as o_file:
        o_file.write('%s\n' % json.dumps(BestParmas))
        o_file.write('cv score %.4f, holdout score %.4f' % (cv_score, holdout_score))
    o_file.close()
    
    end = time.time()
    print('tree %s done, time elapsed %.2f' % (idx, (end - start)))
    
#     if(idx == 0):
#         break

/home/joe/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


running for params, l2 200, lr 0.4, depth 12 done, cv score 0.50198, time elapsed 4.54s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.50212, time elapsed 8.98s
running for params, l2 200, lr 0.4, depth 20 done, cv score 0.50212, time elapsed 13.43s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.50219, time elapsed 18.21s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.50223, time elapsed 23.08s
running for params, l2 200, lr 0.2, depth 20 done, cv score 0.50223, time elapsed 27.79s
running for params, l2 200, lr 0.1, depth 12 done, cv score 0.50387, time elapsed 33.18s
running for params, l2 200, lr 0.1, depth 16 done, cv score 0.50392, time elapsed 38.46s
running for params, l2 200, lr 0.1, depth 20 done, cv score 0.50392, time elapsed 43.78s
running for params, l2 80, lr 0.4, depth 12 done, cv score 0.50280, time elapsed 48.24s
running for params, l2 80, lr 0.4, depth 16 done, cv score 0.50315, time elapsed 52.62s
running for params, l2 80

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.506379, holdout score 0.495816, valid length 45371
fold 1: valid score 0.497112, holdout score 0.495846, valid length 45371
fold 2: valid score 0.500781, holdout score 0.494342, valid length 45371
fold 3: valid score 0.505467, holdout score 0.496163, valid length 45370
fold 4: valid score 0.499972, holdout score 0.495597, valid length 45370

CV score 0.5019, Holdout score 0.4956, Elapsed time: 134.60s

tree 0 done, time elapsed 134.60
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.50474, time elapsed 139.03s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.50482, time elapsed 143.53s
running for params, l2 200, lr 0.4, depth 20 done, cv score 0.50486, time elapsed 147.99s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.50441, time elapsed 152.69s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.50450, time elapsed 157.47s
running for params, l2 200, lr 0.2, depth 20 done, cv score 0.50446, time elapsed 162.13s

running for params, l2 20, lr 0.4, depth 16 done, cv score 0.50957, time elapsed 488.09s
running for params, l2 20, lr 0.4, depth 20 done, cv score 0.50943, time elapsed 492.22s
running for params, l2 20, lr 0.2, depth 12 done, cv score 0.50816, time elapsed 496.65s
running for params, l2 20, lr 0.2, depth 16 done, cv score 0.50831, time elapsed 501.09s
running for params, l2 20, lr 0.2, depth 20 done, cv score 0.50833, time elapsed 505.54s
running for params, l2 20, lr 0.1, depth 12 done, cv score 0.50982, time elapsed 510.51s
running for params, l2 20, lr 0.1, depth 16 done, cv score 0.50987, time elapsed 515.45s
running for params, l2 20, lr 0.1, depth 20 done, cv score 0.50989, time elapsed 520.40s
grid search done, time elapsed 520.40s
best params, {'lambda_l2': 80, 'learning_rate': 0.2, 'max_depth': 12}
fold 0: valid score 0.512345, holdout score 0.502224, valid length 45371
fold 1: valid score 0.503366, holdout score 0.501841, valid length 45371
fold 2: valid score 0.507490, hol

...

running for params, l2 200, lr 0.2, depth 16 done, cv score 0.50770, time elapsed 11109.07s
running for params, l2 200, lr 0.2, depth 20 done, cv score 0.50779, time elapsed 11113.93s
running for params, l2 200, lr 0.1, depth 12 done, cv score 0.50912, time elapsed 11119.10s
running for params, l2 200, lr 0.1, depth 16 done, cv score 0.50913, time elapsed 11124.32s
running for params, l2 200, lr 0.1, depth 20 done, cv score 0.50917, time elapsed 11129.59s
running for params, l2 80, lr 0.4, depth 12 done, cv score 0.50830, time elapsed 11133.85s
running for params, l2 80, lr 0.4, depth 16 done, cv score 0.50788, time elapsed 11138.12s
running for params, l2 80, lr 0.4, depth 20 done, cv score 0.50788, time elapsed 11142.41s
running for params, l2 80, lr 0.2, depth 12 done, cv score 0.50755, time elapsed 11146.94s
running for params, l2 80, lr 0.2, depth 16 done, cv score 0.50742, time elapsed 11151.45s
running for params, l2 80, lr 0.2, depth 20 done, cv score 0.50743, time elapsed 1115

fold 0: valid score 0.505435, holdout score 0.495392, valid length 45371
fold 1: valid score 0.497221, holdout score 0.495807, valid length 45371
fold 2: valid score 0.501872, holdout score 0.495872, valid length 45371
fold 3: valid score 0.505501, holdout score 0.496508, valid length 45370
fold 4: valid score 0.500520, holdout score 0.495931, valid length 45370

CV score 0.5021, Holdout score 0.4959, Elapsed time: 11473.12s

tree 89 done, time elapsed 11473.12
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.50299, time elapsed 11477.35s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.50303, time elapsed 11481.58s
running for params, l2 200, lr 0.4, depth 20 done, cv score 0.50308, time elapsed 11485.82s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.50260, time elapsed 11490.35s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.50250, time elapsed 11494.87s
running for params, l2 200, lr 0.2, depth 20 done, cv score 0.50249, time 

running for params, l2 80, lr 0.1, depth 20 done, cv score 0.50549, time elapsed 11808.10s
running for params, l2 20, lr 0.4, depth 12 done, cv score 0.50504, time elapsed 11812.07s
running for params, l2 20, lr 0.4, depth 16 done, cv score 0.50501, time elapsed 11816.06s
running for params, l2 20, lr 0.4, depth 20 done, cv score 0.50501, time elapsed 11820.07s
running for params, l2 20, lr 0.2, depth 12 done, cv score 0.50331, time elapsed 11824.31s
running for params, l2 20, lr 0.2, depth 16 done, cv score 0.50337, time elapsed 11828.55s
running for params, l2 20, lr 0.2, depth 20 done, cv score 0.50347, time elapsed 11832.91s
running for params, l2 20, lr 0.1, depth 12 done, cv score 0.50512, time elapsed 11837.62s
running for params, l2 20, lr 0.1, depth 16 done, cv score 0.50517, time elapsed 11842.37s
running for params, l2 20, lr 0.1, depth 20 done, cv score 0.50526, time elapsed 11847.11s
grid search done, time elapsed 11847.11s
best params, {'lambda_l2': 20, 'learning_rate': 0

running for params, l2 200, lr 0.1, depth 12 done, cv score 0.51186, time elapsed 12147.64s
running for params, l2 200, lr 0.1, depth 16 done, cv score 0.51177, time elapsed 12153.17s
running for params, l2 200, lr 0.1, depth 20 done, cv score 0.51171, time elapsed 12158.74s
running for params, l2 80, lr 0.4, depth 12 done, cv score 0.50894, time elapsed 12163.22s
running for params, l2 80, lr 0.4, depth 16 done, cv score 0.50892, time elapsed 12167.70s
running for params, l2 80, lr 0.4, depth 20 done, cv score 0.50892, time elapsed 12172.14s
running for params, l2 80, lr 0.2, depth 12 done, cv score 0.50894, time elapsed 12176.90s
running for params, l2 80, lr 0.2, depth 16 done, cv score 0.50899, time elapsed 12181.69s
running for params, l2 80, lr 0.2, depth 20 done, cv score 0.50889, time elapsed 12186.47s
running for params, l2 80, lr 0.1, depth 12 done, cv score 0.51127, time elapsed 12191.75s
running for params, l2 80, lr 0.1, depth 16 done, cv score 0.51131, time elapsed 12196.

fold 0: valid score 0.511183, holdout score 0.500621, valid length 45371
fold 1: valid score 0.501430, holdout score 0.500689, valid length 45371
fold 2: valid score 0.505584, holdout score 0.500060, valid length 45371
fold 3: valid score 0.510494, holdout score 0.500631, valid length 45370
fold 4: valid score 0.505170, holdout score 0.500104, valid length 45370

CV score 0.5068, Holdout score 0.5004, Elapsed time: 12505.17s

tree 97 done, time elapsed 12505.17
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.51071, time elapsed 12509.46s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.51091, time elapsed 12513.71s
running for params, l2 200, lr 0.4, depth 20 done, cv score 0.51091, time elapsed 12517.96s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.51118, time elapsed 12522.45s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.51114, time elapsed 12526.98s
running for params, l2 200, lr 0.2, depth 20 done, cv score 0.51116, time 